In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection, metrics
from sklearn.linear_model import LinearRegression

In [4]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

train['source']='train'
test['source']='test'
df = pd.concat([train, test], ignore_index=True, sort=False)

df.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [5]:
print(train.shape)
print(test.shape) 
print(df.shape) 

(8523, 13)
(5681, 12)
(14204, 13)


In [6]:
df.columns

Index(['Item_Identifier', 'Item_Weight', 'Item_Fat_Content', 'Item_Visibility',
       'Item_Type', 'Item_MRP', 'Outlet_Identifier',
       'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type',
       'Outlet_Type', 'Item_Outlet_Sales', 'source'],
      dtype='object')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
Item_Identifier              14204 non-null object
Item_Weight                  11765 non-null float64
Item_Fat_Content             14204 non-null object
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  10188 non-null object
Outlet_Location_Type         14204 non-null object
Outlet_Type                  14204 non-null object
Item_Outlet_Sales            8523 non-null float64
source                       14204 non-null object
dtypes: float64(4), int64(1), object(8)
memory usage: 1.4+ MB


In [12]:
df.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [14]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [16]:
df['Item_Identifier'].value_counts(dropna=False)

FDU31    10
FDG09    10
FDF08    10
FDI20    10
NCY41    10
FDB38    10
FDE28    10
FDW12    10
DRB25    10
FDT43    10
FDH10    10
FDZ56    10
FDN32    10
FDE16    10
FDQ45    10
FDU40    10
FDT46    10
NCN41    10
FDQ19    10
FDH14    10
FDP15    10
FDZ23    10
FDY40    10
FDS48    10
FDI22    10
FDH41    10
FDP59    10
FDI04    10
FDU11    10
FDT27    10
         ..
FDV13     8
NCT29     8
FDL25     8
DRI49     8
NCY05     8
NCJ05     8
NCI18     8
FDO52     8
FDS34     8
FDC23     8
FDC16     8
FDJ46     8
NCK07     8
FDD52     8
FDI14     8
FDU47     8
FDA20     8
NCW54     7
FDS22     7
FDX49     7
FDL50     7
NCL42     7
FDO33     7
FDM10     7
FDM50     7
FDR51     7
FDH58     7
DRN11     7
FDI46     7
FDM52     7
Name: Item_Identifier, Length: 1559, dtype: int64

In [17]:
df['Item_Weight'].value_counts(dropna=False)

NaN        2439
 17.600     135
 12.150     127
 10.500     123
 13.650     115
 11.800     113
 15.100     111
 19.350     109
 15.700     102
 16.700     100
 15.600     100
 9.800       99
 17.750      99
 20.250      97
 12.600      96
 16.000      95
 15.850      93
 9.300       93
 17.700      93
 10.195      93
 20.700      92
 18.850      92
 14.000      92
 12.500      92
 19.700      91
 15.000      91
 17.850      89
 18.250      89
 16.750      89
 9.600       85
           ... 
 5.465        7
 8.750        7
 8.655        7
 6.480        7
 7.565        7
 7.640        7
 6.520        7
 5.405        7
 7.275        7
 7.070        7
 7.960        7
 6.305        7
 6.405        7
 5.750        7
 6.150        7
 4.555        7
 5.210        7
 5.885        7
 4.920        7
 8.670        7
 8.380        7
 8.350        7
 6.575        7
 5.510        7
 7.485        7
 7.975        7
 8.485        7
 4.615        6
 9.035        6
 7.850        6
Name: Item_Weight, Lengt

In [18]:
df['Item_Fat_Content'].value_counts(dropna=False)

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [19]:
df['Item_Visibility'].value_counts(dropna=False)

0.000000    879
0.076856      3
0.076841      3
0.077290      3
0.077169      3
0.076975      3
0.077011      3
0.076792      3
0.076483      3
0.046899      2
0.159844      2
0.135708      2
0.135944      2
0.072298      2
0.136008      2
0.136896      2
0.203401      2
0.081788      2
0.075049      2
0.058369      2
0.104784      2
0.080131      2
0.072411      2
0.024635      2
0.052069      2
0.079968      2
0.179192      2
0.079806      2
0.121767      2
0.080625      2
           ... 
0.013147      1
0.098790      1
0.073397      1
0.137756      1
0.017937      1
0.105126      1
0.044156      1
0.141639      1
0.094411      1
0.074883      1
0.039236      1
0.044497      1
0.051147      1
0.175061      1
0.039034      1
0.077508      1
0.078943      1
0.077170      1
0.042959      1
0.013173      1
0.069939      1
0.011305      1
0.069042      1
0.012216      1
0.064142      1
0.209684      1
0.019592      1
0.013530      1
0.008772      1
0.066817      1
Name: Item_Visibility, L

In [20]:
df['Item_Type'].value_counts(dropna=False)

Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

In [21]:
df['Item_MRP'].value_counts(dropna=False)

172.0422    8
179.7660    7
127.1020    7
177.9370    7
123.1046    7
166.0526    7
196.5768    7
178.4370    7
175.0370    7
182.1608    7
188.1872    7
110.1544    7
178.2370    7
177.0370    7
113.2834    7
113.1544    7
50.3692     6
152.8366    6
142.0154    6
125.2678    6
119.8124    6
97.0094     6
101.7990    6
100.1016    6
144.6102    6
154.4630    6
140.6154    6
121.3098    6
177.6370    6
194.7110    6
           ..
177.7660    1
151.5708    1
248.5092    1
36.2874     1
85.2908     1
41.9112     1
181.9318    1
149.4392    1
107.5938    1
213.2218    1
244.7460    1
234.0642    1
98.1068     1
228.7352    1
48.3060     1
258.6304    1
251.5382    1
121.7440    1
85.4540     1
127.6362    1
49.2692     1
237.9880    1
236.9932    1
178.4686    1
233.6616    1
176.9712    1
253.6014    1
118.4808    1
230.8010    1
192.0188    1
Name: Item_MRP, Length: 8052, dtype: int64

In [22]:
df['Outlet_Identifier'].value_counts(dropna=False)

OUT027    1559
OUT013    1553
OUT046    1550
OUT049    1550
OUT035    1550
OUT045    1548
OUT018    1546
OUT017    1543
OUT010     925
OUT019     880
Name: Outlet_Identifier, dtype: int64

In [23]:
df['Outlet_Establishment_Year'].value_counts(dropna=False)

1985    2439
1987    1553
1999    1550
1997    1550
2004    1550
2002    1548
2009    1546
2007    1543
1998     925
Name: Outlet_Establishment_Year, dtype: int64

In [24]:
df['Outlet_Size'].value_counts(dropna=False)

Medium    4655
NaN       4016
Small     3980
High      1553
Name: Outlet_Size, dtype: int64

In [25]:
df['Outlet_Location_Type'].value_counts(dropna=False)

Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

In [26]:
df['Outlet_Type'].value_counts(dropna=False)

Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64

In [27]:
df['Item_Outlet_Sales'].value_counts(dropna=False)

NaN           5681
 958.7520       17
 1342.2528      16
 1845.5976      15
 703.0848       15
 1230.3984      14
 1278.3360      14
 1416.8224      13
 759.0120       12
 1438.1280      12
 1211.7560      12
 539.2980       12
 575.2512       12
 1701.7848      11
 1406.1696      11
 2277.0360      11
 2109.2544      11
 852.2240       11
 1537.9980      11
 639.1680       11
 679.1160       11
 1304.9680      11
 1821.6288      10
 1677.8160      10
 2117.2440      10
 3020.0688      10
 3415.5540      10
 2348.9424      10
 1470.0864      10
 918.8040       10
              ... 
 4064.0432       1
 1834.2790       1
 4873.6560       1
 3365.6190       1
 1368.2190       1
 2139.2154       1
 1482.7366       1
 8062.8380       1
 6771.1860       1
 3793.7284       1
 4351.6688       1
 2314.9866       1
 1758.3778       1
 889.5088        1
 1133.1916       1
 4623.3152       1
 2743.7618       1
 346.2160        1
 825.5920        1
 2165.1816       1
 876.1928        1
 53.2640    

In [28]:
df["Item_Weight"].mean()

12.792854228644991

In [37]:
df.pivot_table(values='Item_Weight', index='Item_Identifier')

,Item_Weight
Item_Identifier,
DRA12,11.600
DRA24,19.350
DRA59,8.270
DRB01,7.390
DRB13,6.115
DRB24,8.785
DRB25,12.300
DRB48,16.750
DRC01,5.920


In [8]:
weight_mean = df.pivot_table(values='Item_Weight', index='Item_Identifier')

In [9]:
def check_missing(cols):
    Weight = cols[0]
    Identifier = cols[1]
    
    if pd.isnull(Weight):
        return weight_mean['Item_Weight'][weight_mean.index == Identifier]
    else:
        return Weight


df['Item_Weight'] = df[['Item_Weight','Item_Identifier']].apply(check_missing, axis=1).astype(float)

df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [57]:
df.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=lambda x:x.mode())

Outlet_Type,Grocery Store,Supermarket Type1,Supermarket Type2,Supermarket Type3
Outlet_Size,Small,Small,Medium,Medium


In [10]:
size_mode = df.pivot_table(values='Outlet_Size', columns='Outlet_Type', aggfunc=lambda x:x.mode())

In [13]:
def check_missing_2(cols):
    Size = cols[0]
    Type = cols[1]
    if pd.isnull(Size):
        return size_mode.loc['Outlet_Size'][size_mode.columns == Type][0]
    else:
        return Size
    
    

df['Outlet_Size'] = df[['Outlet_Size','Outlet_Type']].apply(check_missing_2,axis=1)

df.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
source                          0
dtype: int64

In [65]:
df.pivot_table(values='Item_Visibility', index='Item_Identifier')

,Item_Visibility
Item_Identifier,
DRA12,0.034938
DRA24,0.045646
DRA59,0.133384
DRB01,0.079736
DRB13,0.006799
DRB24,0.020596
DRB25,0.079407
DRB48,0.023973
DRC01,0.020653


In [14]:
visib_mean = df.pivot_table(values='Item_Visibility', index='Item_Identifier')

In [15]:
def check_zeros(cols):
    visib = cols[0]
    item = cols[1]
    if visib == 0:
        return visib_mean['Item_Visibility'][visib_mean.index == item]
    else:
        return visib

df['Item_Visibility'] = df[['Item_Visibility','Item_Identifier']].apply(check_zeros,axis=1).astype(float)

In [69]:
df['Item_Visibility'].value_counts(dropna=False)

0.040627    6
0.008510    4
0.016397    4
0.054325    3
0.094969    3
0.009000    3
0.030341    3
0.076975    3
0.077169    3
0.059170    3
0.077011    3
0.042382    3
0.106538    3
0.014291    3
0.076483    3
0.048620    3
0.034135    3
0.118016    3
0.050969    3
0.018757    3
0.018219    3
0.068954    3
0.057059    3
0.077290    3
0.028608    3
0.076792    3
0.096667    3
0.076856    3
0.076841    3
0.009780    3
           ..
0.043810    1
0.017557    1
0.064272    1
0.213125    1
0.035651    1
0.148682    1
0.116366    1
0.047743    1
0.021021    1
0.016824    1
0.020551    1
0.018411    1
0.022976    1
0.063916    1
0.122305    1
0.017153    1
0.093813    1
0.065618    1
0.053765    1
0.038271    1
0.105812    1
0.034542    1
0.013147    1
0.098790    1
0.073397    1
0.137756    1
0.044149    1
0.048332    1
0.043202    1
0.115072    1
Name: Item_Visibility, Length: 13688, dtype: int64

In [70]:
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,14204.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.793380,0.069710,141.004977,1997.830681,2181.288914
std,4.651716,0.049728,62.086938,8.371664,1706.499616
min,4.555000,0.003575,31.290000,1985.000000,33.290000
25%,8.710000,0.031145,94.012000,1987.000000,834.247400
50%,12.600000,0.057194,142.247000,1999.000000,1794.331000
75%,16.750000,0.096930,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [16]:
df['Item_Type_Combined'] = df['Item_Identifier'].apply(lambda x: x[0:2])
df['Item_Type_Combined'] = df['Item_Type_Combined'].map({'FD':'Food', 'DR':'Drinks', 'NC':'Non-Consumable'})

df['Item_Type_Combined'].value_counts()

Food              10201
Non-Consumable     2686
Drinks             1317
Name: Item_Type_Combined, dtype: int64

In [17]:
df['Item_Fat_Content'] = df['Item_Fat_Content'].replace({'LF':'Low Fat', 'reg':'Regular', 'low fat':'Low Fat'})
df['Item_Fat_Content'].value_counts()

Low Fat    9185
Regular    5019
Name: Item_Fat_Content, dtype: int64

In [18]:
df['Outlet_Years'] = 2013 - df['Outlet_Establishment_Year']
df.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Outlet_Years
count,14204.000000,14204.000000,14204.000000,14204.000000,8523.000000,14204.000000
mean,12.793380,0.069710,141.004977,1997.830681,2181.288914,15.169319
std,4.651716,0.049728,62.086938,8.371664,1706.499616,8.371664
min,4.555000,0.003575,31.290000,1985.000000,33.290000,4.000000
25%,8.710000,0.031145,94.012000,1987.000000,834.247400,9.000000
50%,12.600000,0.057194,142.247000,1999.000000,1794.331000,14.000000
75%,16.750000,0.096930,185.855600,2004.000000,3101.296400,26.000000
max,21.350000,0.328391,266.888400,2009.000000,13086.964800,28.000000


In [19]:
LE = LabelEncoder()

df['Outlet'] = LE.fit_transform(df['Outlet_Identifier'])

x = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for i in x:
    df[i] = LE.fit_transform(df[i])

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 16 columns):
Item_Identifier              14204 non-null object
Item_Weight                  14204 non-null float64
Item_Fat_Content             14204 non-null int64
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Outlet_Size                  14204 non-null int64
Outlet_Location_Type         14204 non-null int64
Outlet_Type                  14204 non-null int64
Item_Outlet_Sales            8523 non-null float64
source                       14204 non-null object
Item_Type_Combined           14204 non-null int64
Outlet_Years                 14204 non-null int64
Outlet                       14204 non-null int64
dtypes: float64(4), int64(8), object(4)
memory usage: 1.7+ MB


In [20]:
df = pd.get_dummies(df, columns = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 35 columns):
Item_Identifier              14204 non-null object
Item_Weight                  14204 non-null float64
Item_Visibility              14204 non-null float64
Item_Type                    14204 non-null object
Item_MRP                     14204 non-null float64
Outlet_Identifier            14204 non-null object
Outlet_Establishment_Year    14204 non-null int64
Item_Outlet_Sales            8523 non-null float64
source                       14204 non-null object
Outlet_Years                 14204 non-null int64
Item_Fat_Content_0           14204 non-null uint8
Item_Fat_Content_1           14204 non-null uint8
Outlet_Location_Type_0       14204 non-null uint8
Outlet_Location_Type_1       14204 non-null uint8
Outlet_Location_Type_2       14204 non-null uint8
Outlet_Size_0                14204 non-null uint8
Outlet_Size_1                14204 non-null uint8
Outlet_Size_2                1

In [79]:
df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Outlet_Years,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,train,14,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,train,4,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.2700,train,14,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.017834,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,train,15,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.009780,Household,53.8614,OUT013,1987,994.7052,train,26,...,0,1,0,0,0,0,0,0,0,0


In [21]:
df.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

train = df.loc[df['source']=="train"]
test = df.loc[df['source']=="test"]

train.drop(['source'],axis=1,inplace=True)
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)

C:\Python\Anaconda3\lib\site-packages\pandas\core\frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [22]:
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
predictors = [x for x in train.columns if x not in [target]+IDcol]

In [25]:
LR_Model = LinearRegression(normalize=True)


LR_Model.fit(train[predictors], train[target])
train_predictions = LR_Model.predict(train[predictors])

cv_score = model_selection.cross_val_score(LR_Model, train[predictors], train[target], cv=20, scoring='neg_mean_squared_error')
cv_score = np.sqrt(np.abs(cv_score))
    
accuracy = LR_Model.score(train[predictors], train[target])
    
print("\nModel Report")
print("RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(train[target].values, train_predictions)))
print("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
print("Accuracy : %.4g" % (accuracy*100),"%")
    
test[target] = LR_Model.predict(test[predictors])
    
IDcol.append(target)
submission = pd.DataFrame({ x: test[x] for x in IDcol})
submission.to_csv('LR_Model.csv', index=False)



Model Report
RMSE : 1127
CV Score : Mean - 1129 | Std - 43.37 | Min - 1074 | Max - 1209
Accuracy : 56.34 %


C:\Python\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
